In [1]:
import numpy as np 
import pdb 

In [31]:
A = np.random.rand(1000,100)*1.0
B = np.random.rand(1000,200)*100.0
U, S, V_ = np.linalg.svd(B, full_matrices=False)
T = U.dot(np.diag(S))
X_ideal, _, _, _ = np.linalg.lstsq(A,T)

In [32]:

ddL = A.T.dot(A)
m, n = X_ideal.shape
V = np.zeros([m,n])
# X = X_ideal[:]
X_round = np.round(X_ideal)
V_round = X_round - X_ideal 
V = V_round

for k in range(1):
    dL = (A.T.dot(A)).dot(X_ideal + V_round) - A.T.dot(T)
    

# for k in range(1):
#     dL = (A.T.dot(A)).dot(X_ideal + V) - A.T.dot(T)
#     for j in range (n):
#         for i in range(m):
#             X_low = X[:]
#             X_hi = X[:]
#             X_low[i,j] = np.floor(X_ideal[i, j])
#             X_hi[i,j] = np.ceil(X_ideal[i,j])
#             L_low = 0.5*np.linalg.norm(A.dot(X_low) - T)**2
#             L_hi = 0.5*np.linalg.norm(A.dot(X_hi) - T)**2
#             if L_low < L_hi: 
#                 X[i, j] = X_low[i, j]
#             elif L_low > L_hi:
#                 X[i, j] = X_hi[i, j]
#             i = m - i_ - 1 
#             j = n - j_ -1 
#             V_low = V[:]
#             V_low[i, j] = np.floor(X_ideal[i, j]) - X_ideal[i, j]
#             V_hi = V[:]
#             V_hi[i, j] = np.ceil(X_ideal[i, j]) - X_ideal[i, j]
#             dL_low = np.trace(dL.T.dot(V_low)) + 0.5*np.trace((ddL.dot(V_low)).T.dot(V_low))
#             dL_hi = np.trace(dL.T.dot(V_hi)) + 0.5*np.trace((ddL.dot(V_hi)).T.dot(V_hi))
#             if dL_low < dL_hi: 
#                 V[i, j] = V_low[i, j]
#             elif dL_low > dL_hi:
#                 V[i, j] = V_hi[i, j]
    #         print('%r of % done' % (i + j*m + 1, m*n))

In [33]:
# X = X_ideal + V
X_round = np.round(X_ideal)
loss = 0.5*np.linalg.norm((A.dot(X)).dot(V_) - B)**2
loss_round = 0.5*np.linalg.norm((A.dot(X_round)).dot(V_) - B)**2
print('My Loss:', loss)
print('Naive Loss:', loss_round)

My Loss: 138982810.647
Naive Loss: 138982810.647


In [22]:
np.trace(dL.T.dot(X_ideal))

4.2403888808661161e-08

In [19]:
X

array([[-61.,  23., -18.,   1.,   4.,  17.,   3.,  -3.,  13.,  -2.,  26.,
         16.,  13.,   5., -11.,  -1.,  -2.,   6.,   4.,  11.],
       [-66.,  -2.,   4.,  11., -19.,  -3.,  22.,   3.,   5., -11., -21.,
          7.,  -3.,  10.,   3., -10.,   1.,  15.,   3.,  -1.],
       [-58., -32.,  25.,  12., -19., -12.,  19., -17.,  18.,  19.,   2.,
         15.,  -7.,  10.,  -1., -13.,  -9.,  -9.,  -2.,  -2.],
       [-50., -24., -17.,   6.,  37., -31.,  20., -11.,  11.,   6., -12.,
        -13.,  11., -14.,   6.,   0.,   7.,  -8., -13.,  -4.],
       [  1.,   3., -22., -13.,  12., -10., -19.,  17.,  15., -13.,   1.,
         16., -13., -14.,   0.,   8.,  14.,  -7., -13.,  -1.],
       [-50.,   0.,   3., -13.,   0.,  39., -14.,  -4., -14.,   6.,   9.,
        -23., -11.,   3.,   2.,  -3., -14.,   3.,  -2.,  -6.],
       [-80.,  14.,  12.,  15.,  -6., -10.,   3.,  12., -12.,  18.,  26.,
        -11.,  32.,  13.,   8.,  -4.,   8.,   5.,   8.,  12.],
       [-29., -18., -36.,  -7.,  33.,  16

In [20]:
X_round

array([[-62.,  22., -18.,   1.,   4.,  17.,   3.,  -3.,  13.,  -2.,  25.,
         16.,  12.,   5., -11.,  -1.,  -3.,   5.,   4.,  11.],
       [-67.,  -2.,   3.,  11., -19.,  -3.,  22.,   3.,   4., -11., -21.,
          7.,  -3.,  10.,   3., -10.,   0.,  15.,   2.,  -2.],
       [-58., -33.,  25.,  11., -19., -12.,  18., -17.,  17.,  18.,   1.,
         14.,  -8.,  10.,  -2., -13.,  -9.,  -9.,  -3.,  -2.],
       [-50., -24., -17.,   6.,  37., -32.,  20., -12.,  10.,   5., -13.,
        -14.,  11., -14.,   5.,  -0.,   6.,  -9., -14.,  -4.],
       [  1.,   3., -22., -13.,  12., -11., -19.,  17.,  14., -13.,   0.,
         16., -14., -15.,  -0.,   7.,  14.,  -7., -13.,  -2.],
       [-51.,  -0.,   3., -13.,  -1.,  38., -15.,  -4., -15.,   5.,   9.,
        -23., -11.,   2.,   2.,  -3., -14.,   2.,  -3.,  -7.],
       [-80.,  13.,  12.,  15.,  -6., -11.,   2.,  12., -13.,  18.,  26.,
        -12.,  32.,  12.,   8.,  -4.,   8.,   5.,   7.,  12.],
       [-29., -19., -37.,  -8.,  32.,  16

In [21]:
X_ideal

array([[-61.55801298,  22.17325017, -18.40437087,   0.60488248,
          3.74337513,  16.72365393,   2.79156629,  -3.38240778,
         12.88392325,  -2.22819074,  25.09074991,  15.70338883,
         12.43600441,   4.68726073, -11.01972697,  -1.09735278,
         -2.90309469,   5.17401219,   3.75082026,  10.54627833],
       [-66.57209632,  -2.26044067,   3.03974567,  10.6370506 ,
        -19.41435552,  -3.48854013,  21.59453364,   2.61537717,
          4.35344265, -11.07113351, -21.18376243,   6.58790328,
         -3.28970358,   9.5952105 ,   2.74617112, -10.09464122,
          0.12338762,  14.80253537,   2.12744681,  -1.91798091],
       [-58.06867326, -32.85752366,  24.68844444,  11.40431801,
        -19.08347967, -12.11439911,  18.30684964, -17.42444362,
         17.16073628,  18.04680908,   1.40477224,  14.41693095,
         -7.58077803,   9.59161566,  -1.92971244, -13.22399616,
         -9.31741281,  -9.03205754,  -2.93736507,  -2.08433488],
       [-50.30722512, -24.11834881, -